In [ ]:
#Importing necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#cleaning the data
df=pd.read_csv("/kaggle/input/fraud-detection/Fraud.csv")
df.dropna(how="any",inplace=True)
df.head()

In [ ]:
#checking for na
df.isna().any()

In [ ]:
#correlation matrix to find suitable features
matrix=df.corr()
sns.heatmap(matrix,annot=True,cmap='summer')

In [ ]:
#number of fraud cases
fraudf=df[df["isFraud"]==1]
isfraud=fraudf['isFraud'].count()
fraudf2=df[df["isFraud"]==0 ]
isntfraud=fraudf2["isFraud"].count()
print("Fraud cases: ",isfraud)
print("Normal cases: ",isntfraud)

In [ ]:
#Fraud percentage
print("Fraud case fraction",round(isfraud/(isfraud+isntfraud),5))

In [ ]:
#creating training dataset
from sklearn.model_selection import train_test_split
xcols=["amount","oldbalanceDest","newbalanceDest","oldbalanceOrg","newbalanceOrig"]
x=df[xcols]
y=df["isFraud"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=50,shuffle=True)

In [ ]:
#Scaling data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

#Training the model
import tensorflow as tf
tf.random.set_seed(10)

modelf = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

modelf.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=tf.keras.metrics.Recall()
)

modelf.fit(x_train_scaled, y_train, epochs=5)

#predicting test set values
import numpy as np
predictions=modelf.predict(x_test_scaled)
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]
print(predictions)


In [ ]:
#performance of the model
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, prediction_classes)
print(cm)
sns.heatmap(cm,annot=True,cmap='summer')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')
